In [ ]:
import sqlalchemy

# connection
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# declare a mapping

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DATETIME, UniqueConstraint

Base = declarative_base()

class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    channel = Column(String)
    title = Column(String)
    guid = Column(String)
    link = Column(String)
    published = Column(DATETIME)
    
    UniqueConstraint(channel, guid)

    def __repr__(self):
        return "<Post(id='%s', channel='%s', title='%s', guid='%s', link='%s', published='%s')>" % (self.id, self.channel, self.title, self.guid, self.link, self.published)

Post.__table__    

class Channel(Base):
    __tablename__ = 'channels'
    
    id = Column(Integer, primary_key=True)
    type = Column(String)
    title = Column(String, unique=True)
    url = Column(String)
    etag = Column(String, nullable=True)
    modified = Column(DATETIME, nullable=True)
    
#     UniqueConstraint(type, title)
    
    def __repr__(self):
        return "<Channel(id='%s', type='%s', title='%s', url='%s', etag='%s', modified='%s')>" % (self.id, self.type, self.title, self.url, self.etag, self.modified)
    


# create a schema

Base.metadata.create_all(engine)

# creating a session

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2018-06-10 16:43:49,947 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-06-10 16:43:49,949 INFO sqlalchemy.engine.base.Engine ()
2018-06-10 16:43:49,951 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-06-10 16:43:49,954 INFO sqlalchemy.engine.base.Engine ()
2018-06-10 16:43:49,958 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("posts")
2018-06-10 16:43:49,962 INFO sqlalchemy.engine.base.Engine ()
2018-06-10 16:43:49,968 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("channels")
2018-06-10 16:43:49,970 INFO sqlalchemy.engine.base.Engine ()
2018-06-10 16:43:49,975 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE posts (
	id INTEGER NOT NULL, 
	channel VARCHAR, 
	title VARCHAR, 
	guid VARCHAR, 
	link VARCHAR, 
	published DATETIME, 
	PRIMARY KEY (id), 
	UNIQUE (channel, guid)
)


2018-06-10 16:43:49,983 INFO sqlalchemy.engine.base.Engine ()
2018-06-10 16:43:49,987 INFO sqlalc

In [12]:
RSS_CHANNELS = [
    Channel(
        type='RSS',
        title='Yarmini',
        url='http://yarmini.com/feed'        
    ),
    Channel(
        type='RSS',
        title= '이동진의 빨간책방',
        url= 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690',
    )
]

from sqlalchemy import exc

for channel in RSS_CHANNELS:
    try:
        session.add(channel)
        session.commit()
    except exc.IntegrityError as e:
        print (e)
        session.rollback()


2018-06-10 16:43:50,037 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:43:50,044 INFO sqlalchemy.engine.base.Engine INSERT INTO channels (type, title, url, etag, modified) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:50,048 INFO sqlalchemy.engine.base.Engine ('RSS', 'Yarmini', 'http://yarmini.com/feed', None, None)
2018-06-10 16:43:50,054 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-10 16:43:50,058 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:43:50,063 INFO sqlalchemy.engine.base.Engine INSERT INTO channels (type, title, url, etag, modified) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:50,069 INFO sqlalchemy.engine.base.Engine ('RSS', '이동진의 빨간책방', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690', None, None)
2018-06-10 16:43:50,072 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
channels = session.query(Channel).all()
channels

2018-06-10 16:43:50,091 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:43:50,103 INFO sqlalchemy.engine.base.Engine SELECT channels.id AS channels_id, channels.type AS channels_type, channels.title AS channels_title, channels.url AS channels_url, channels.etag AS channels_etag, channels.modified AS channels_modified 
FROM channels
2018-06-10 16:43:50,106 INFO sqlalchemy.engine.base.Engine ()


[<Channel(id='1', type='RSS', title='Yarmini', url='http://yarmini.com/feed', etag='None', modified='None')>,
 <Channel(id='2', type='RSS', title='이동진의 빨간책방', url='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690', etag='None', modified='None')>]

In [14]:
import feedparser
import pprint
from time import mktime
from datetime import datetime

pp = pprint.PrettyPrinter()


print(channels)

for channel in channels:
    print (channel)
    
    feed = feedparser.parse(channel.url, etag=channel.etag)
    feed_title = feed['feed']['title']
    
    if 'etag' in feed:
        feed_etag = feed['etag']
        print (feed_etag)
    
    if 'modified_parsed' in feed:
        feed_modified = datetime.fromtimestamp(mktime(feed.modified_parsed))
        print (feed_modified)
    
#     print (feed)

    posts = []
    
    print (len(feed.entries))

    for post in feed.entries:
    #     print(post)

        published_datetime = datetime.fromtimestamp(mktime(post.published_parsed))
    #     print (type(published))

        posts.append(Post(
            channel=channel.title, title=post.title, guid=post.id, link=post.link, published=published_datetime
        ))

#     pp.pprint(posts)

    channel.etag = feed_etag

channels
print (channels[0].etag)
feed = feedparser.parse(channels[0].url, channels[0].etag)
print (len(feed.entries))


[<Channel(id='1', type='RSS', title='Yarmini', url='http://yarmini.com/feed', etag='None', modified='None')>, <Channel(id='2', type='RSS', title='이동진의 빨간책방', url='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690', etag='None', modified='None')>]
<Channel(id='1', type='RSS', title='Yarmini', url='http://yarmini.com/feed', etag='None', modified='None')>
"4ff7555f03c21da8b40706c9cb419a87-gzip"
2018-04-15 00:45:23
3
<Channel(id='2', type='RSS', title='이동진의 빨간책방', url='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690', etag='None', modified='None')>
328
"4ff7555f03c21da8b40706c9cb419a87-gzip"
3


In [15]:
session.add_all(posts)
session.commit()

2018-06-10 16:43:51,155 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,157 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '273회 - 지리의 힘 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10116151', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10116151', '2018-06-05 22:00:00.000000')
2018-06-10 16:43:51,159 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,162 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '272회 - 경제학 콘서트 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10090995', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10090995', '2018-05-29 22:00:00.000000')
2018-06-10 16:

2018-06-10 16:43:51,240 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,242 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '256회 - 달콤한 노래 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9683462', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9683462', '2018-02-06 16:07:39.000000')
2018-06-10 16:43:51,245 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,248 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '255회 - 달콤한 노래 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9655656', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9655656', '2018-01-30 16:31:17.000000')
2018-06-10 16:43:5

2018-06-10 16:43:51,327 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,329 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '239회 - 나의 눈부신 친구 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9207046', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9207046', '2017-09-26 15:01:50.000000')
2018-06-10 16:43:51,332 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,334 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '238회 - 여기가 아니면 어디라도 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9176672', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9176672', '2017-09-19 15:41:13.000000')
2018-06-1

2018-06-10 16:43:51,426 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,431 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '223회 - 모던 팝 스토리 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979126', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979126', '2017-05-23 16:00:42.000000')
2018-06-10 16:43:51,433 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,436 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '222회 - 13.67 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979124', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979124', '2017-05-17 05:55:01.000000')
2018-06-10 16:43:

2018-06-10 16:43:51,535 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,537 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '207회 - 토니와 수잔 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979107', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979107', '2017-01-31 15:32:53.000000')
2018-06-10 16:43:51,541 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:51,542 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '206회 - 지금 다시, 헌법 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979106', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979106', '2017-01-24 17:17:11.000000')
2018-06-10 16:4

...

2018-06-10 16:43:52,652 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,654 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '56화 - 2013 송년특집(with 이적 and 2013 빨책 베스트) 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978679', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978679', '2013-12-17 15:06:54.000000')
2018-06-10 16:43:52,658 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,665 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '55화 - 2013 송년특집(with 이적) 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978678', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978678', '2013-12-1

2018-06-10 16:43:52,746 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '41회- 연애의 시대 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978663', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978663', '2013-09-12 06:38:24.000000')
2018-06-10 16:43:52,753 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,756 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '40회- 색채가 없는 다자키 쓰쿠루와 그가 순례를 떠난 해 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978662', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978662', '2013-09-12 06:38:22.000000')
2018-06-10 16:43:52,757 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2

2018-06-10 16:43:52,848 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,850 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '29회- 김 박사는 누구인가 1부[PDF]', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978640', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978640', '2013-09-12 06:37:59.000000')
2018-06-10 16:43:52,853 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,855 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '28회- 예감은 틀리지 않는다 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978639', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978639', '2013-09-12 06:37:58.000000')
2018-06-

2018-06-10 16:43:52,954 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,965 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '20회- 파이 이야기', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978618', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978618', '2013-09-12 06:37:44.000000')
2018-06-10 16:43:52,969 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:52,971 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '19회- 철학자와 늑대', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978617', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978617', '2013-09-12 06:37:42.000000')
2018-06-10 16:43:52,974 INF

2018-06-10 16:43:53,052 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '7회- 음식으로 보는 예술과 사회(with 소설가 김중혁)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978574', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978574', '2013-09-12 06:37:14.000000')
2018-06-10 16:43:53,060 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:43:53,063 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '6회- 대가의 소설들(with 소설가 김중혁)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978572', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978572', '2013-09-12 06:37:12.000000')
2018-06-10 16:43:53,065 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?,

In [16]:
session.query(Post).all()

2018-06-10 16:43:53,129 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:43:53,137 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.channel AS posts_channel, posts.title AS posts_title, posts.guid AS posts_guid, posts.link AS posts_link, posts.published AS posts_published 
FROM posts
2018-06-10 16:43:53,138 INFO sqlalchemy.engine.base.Engine ()


[<Post(id='1', channel='이동진의 빨간책방', title='273회 - 지리의 힘 1부', guid='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10116151', link='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10116151', published='2018-06-05 22:00:00')>,
 <Post(id='2', channel='이동진의 빨간책방', title='272회 - 경제학 콘서트 2부', guid='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10090995', link='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10090995', published='2018-05-29 22:00:00')>,
 <Post(id='3', channel='이동진의 빨간책방', title='271회 - 경제학 콘서트 1부', guid='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10081176', link='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/10081176', published='2018-05-26 0

In [17]:
session.add_all(posts)
session.commit()
len(session.query(Post).all())

2018-06-10 16:43:53,189 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-10 16:43:53,196 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:43:53,200 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.channel AS posts_channel, posts.title AS posts_title, posts.guid AS posts_guid, posts.link AS posts_link, posts.published AS posts_published 
FROM posts
2018-06-10 16:43:53,202 INFO sqlalchemy.engine.base.Engine ()


328

In [22]:
from sqlalchemy import exc

print(channels)

for channel in channels:
    print (channel)
    
    feed = feedparser.parse(channel.url)
    feed_title = feed['feed']['title']
#     print (feed)

    posts = []

    for post in feed.entries:
    #     print(post)

        published_datetime = datetime.fromtimestamp(mktime(post.published_parsed))
    #     print (type(published))

        posts.append(Post(
            channel=channel.title, title=post.title, guid=post.id, link=post.link, published=published_datetime
        ))

for post in posts:
    try:
        session.add(post)
        session.commit()
    except exc.IntegrityError as e:
        print (e)
        session.rollback()
len(session.query(Post).all())

[<Channel(id='1', type='RSS', title='Yarmini', url='http://yarmini.com/feed', etag='"4ff7555f03c21da8b40706c9cb419a87-gzip"', modified='None')>, <Channel(id='2', type='RSS', title='이동진의 빨간책방', url='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690', etag='"4ff7555f03c21da8b40706c9cb419a87-gzip"', modified='None')>]
<Channel(id='1', type='RSS', title='Yarmini', url='http://yarmini.com/feed', etag='"4ff7555f03c21da8b40706c9cb419a87-gzip"', modified='None')>
<Channel(id='2', type='RSS', title='이동진의 빨간책방', url='http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/174690', etag='"4ff7555f03c21da8b40706c9cb419a87-gzip"', modified='None')>
2018-06-10 16:45:52,999 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:53,000 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '273회 - 지리의 힘 1부', 'http://api.podty

2018-06-10 16:45:53,105 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:53,108 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '266회 - 달의 궁전 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9942694', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9942694', '2018-04-17 16:37:19.000000')
2018-06-10 16:45:53,113 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '266회 - 달의 궁전 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9942694', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9942694', '

2018-06-10 16:45:53,217 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '259회 - 제5도살장 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9760157', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9760157', '2018-02-28 02:58:30.000000')]
2018-06-10 16:45:53,222 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:45:53,224 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:53,226 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '258회 - 알아두면 마음편한 인생선택 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9734062', 'http://api.podty.me/api/v1/s

2018-06-10 16:45:53,328 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:53,330 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '251회 - 쇼코의 미소 1부 with 최은영 작가', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9516557', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9516557', '2017-12-19 15:28:01.000000')
2018-06-10 16:45:53,332 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '251회 - 쇼코의 미소 1부 with 최은영 작가', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9516557', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be098

2018-06-10 16:45:53,438 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '244회 - 나를 보내지 마 2부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9341554', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9341554', '2017-10-31 16:00:00.000000')]
2018-06-10 16:45:53,445 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:45:53,449 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:53,452 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '243회 - 나를 보내지 마 1부', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/9316505', 'http://api.podty.me/api/v1/shar

...

2018-06-10 16:45:57,764 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:57,767 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '22회- 너의 목소리가 들려', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979246', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979246', '2013-09-12 06:37:48.000000')
2018-06-10 16:45:57,770 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '22회- 너의 목소리가 들려', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979246', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8979246', '

2018-06-10 16:45:57,874 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '17회- 송년특집2탄(임자 어워드)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978614', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978614', '2013-09-12 06:37:38.000000')]
2018-06-10 16:45:57,880 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:45:57,882 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:57,884 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '16회- 송년특집1탄(with 루시드폴)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978613', 'http://api.podty.me/api/v1

2018-06-10 16:45:57,987 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:57,989 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '11회- 1st 북콘서트(가을방학,소규모아카시아밴드)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978588', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978588', '2013-09-12 06:37:24.000000')
2018-06-10 16:45:57,991 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '11회- 1st 북콘서트(가을방학,소규모아카시아밴드)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978588', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0

2018-06-10 16:45:58,112 INFO sqlalchemy.engine.base.Engine ROLLBACK
(sqlite3.IntegrityError) UNIQUE constraint failed: posts.channel, posts.guid [SQL: 'INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)'] [parameters: ('이동진의 빨간책방', '5회- 사라진 직업들(with 소설가 김중혁)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978569', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978569', '2013-09-12 06:37:10.000000')]
2018-06-10 16:45:58,117 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-10 16:45:58,121 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (channel, title, guid, link, published) VALUES (?, ?, ?, ?, ?)
2018-06-10 16:45:58,123 INFO sqlalchemy.engine.base.Engine ('이동진의 빨간책방', '4회- 영화가 된 소설들 vol2(with 소설가 김중혁)', 'http://api.podty.me/api/v1/share/cast/b4556578e4f143d676c7acb660ca358bd58568a192855be0988d133a6bfceaaa/8978568', 'http://api

328